In [1]:
import scapy
from scapy.all import sniff
from collections import Counter

In [2]:
def handle_packet(packet):
    """
    Function to run on each packet.
    """
    print(packet.summary())

sniff(prn=handle_packet, count=10)

Ether / IPv6 / TCP 2600:4040:402f:ac00:b99a:8e6:968a:3cad:34096 > 2606:4700::6812:17d2:https A / Raw
Ether / IPv6 / TCP 2606:4700::6812:17d2:https > 2600:4040:402f:ac00:b99a:8e6:968a:3cad:34096 A
Ether / IPv6 / UDP 2600:4040:402f:ac00:b99a:8e6:968a:3cad:56415 > 2606:4700::6812:3a1:https / Raw
Ether / IPv6 / UDP 2600:4040:402f:ac00:b99a:8e6:968a:3cad:56415 > 2606:4700::6812:3a1:https / Raw
Ether / IPv6 / UDP 2600:4040:402f:ac00:b99a:8e6:968a:3cad:56415 > 2606:4700::6812:3a1:https / Raw
Ether / IPv6 / UDP 2600:4040:402f:ac00:b99a:8e6:968a:3cad:56415 > 2606:4700::6812:3a1:https / Raw
Ether / IPv6 / UDP 2606:4700::6812:3a1:https > 2600:4040:402f:ac00:b99a:8e6:968a:3cad:56415 / Raw
Ether / IP / TCP 52.86.220.33:https > 192.168.1.158:34217 PA / Raw
Ether / IP / TCP 192.168.1.158:34217 > 52.86.220.33:https A
Ether / IPv6 / UDP 2606:4700::6812:3a1:https > 2600:4040:402f:ac00:b99a:8e6:968a:3cad:56415 / Raw


<Sniffed: TCP:4 UDP:6 ICMP:0 Other:0>

In [3]:
def handle_packet0(packet):
    print(f"Source: {packet[IP].src}, Destination: {pacet[IP].dst}, Protocol: {packet[IP],proto}, Length: {len(packet)}")

In [4]:
ip_counter = Counter()

def handle_packet1(packet):
    ip_counter[packet[IP].src] += 1
    ip_counter[packet[IP].dst] += 1